In [2]:
# Import Required Libraries
import pandas as pd
import random

In [5]:
# Number of data points
n = 10000

# Simulate data collection
def generate_data(n):
    data = {
        'Humidity (%)': [random.uniform(0, 100) for _ in range(n)],
        'Nitrogen Levels (mg/kg)': [random.uniform(0, 200) for _ in range(n)],
        'Ambient Temperature (°C)': [random.uniform(-10, 40) for _ in range(n)],
        'Presence of Sunlight': [random.choice([0, 1]) for _ in range(n)],
        'Plant Health': []
    }
    
    # A simple rule-based approach to determine plant health based on the simulated data
    for i in range(n):
        if (50 <= data['Humidity (%)'][i] <= 70) and (100 <= data['Nitrogen Levels (mg/kg)'][i] <= 150) and (15 <= data['Ambient Temperature (°C)'][i] <= 25) and data['Presence of Sunlight'][i] == 1:
            data['Plant Health'].append('Healthy')
        elif (40 <= data['Humidity (%)'][i] <= 80) and (80 <= data['Nitrogen Levels (mg/kg)'][i] <= 170) and (10 <= data['Ambient Temperature (°C)'][i] <= 30):
            data['Plant Health'].append('Moderate')
        else:
            data['Plant Health'].append('Unhealthy')
    
    return pd.DataFrame(data)

# Generate dataset
df = generate_data(n)

# Save to CSV
df.to_csv('plant_health_dataset.csv', index=False)


In [6]:
df.head()

,Humidity (%),Nitrogen Levels (mg/kg),Ambient Temperature (°C),Presence of Sunlight,Plant Health
0,37.487489,79.049495,5.336299,0,Unhealthy
1,29.273504,20.328702,39.486602,0,Unhealthy
2,8.165586,70.341813,-1.484463,1,Unhealthy
3,35.961232,61.570803,16.185644,1,Unhealthy
4,1.398062,180.731597,-4.244504,0,Unhealthy
